In [ ]:
from selenium import webdriver as web
from bs4 import BeautifulSoup
import time
import pandas
import json
import unidecode

In [ ]:
# Acessar a página de cotações
option = web.ChromeOptions()
#option.headless = True
driver = web.Chrome(options=option)

In [ ]:
driver.get("https://economia.awesomeapi.com.br/xml/available/uniq")
time.sleep(3)

main_element = driver.find_element_by_class_name("pretty-print")
html = main_element.get_attribute('outerHTML')
soup = BeautifulSoup(html, 'html.parser')




In [ ]:
class_line = soup.find_all('div', attrs={'class':"line"})
class_line

## Pegando os códigos de cada moeda

In [ ]:
moedas = []
for line in class_line:
    moedas.append(line.find("span").get_text().strip("<>"))

In [ ]:
moedas.pop(0)
moedas.pop(-1)

## Pegando os países das cotações da moeda

In [ ]:
paises = []
for line in class_line:
    if len(line.find_all('span'))>1:
        paises.append(str(unidecode.unidecode(line.find_all("span")[1].get_text())))
    

In [ ]:
paises.pop(0)

In [ ]:
driver.get("https://economia.awesomeapi.com.br/xml/available")
time.sleep(5)

main_element = driver.find_element_by_class_name("opened")
html = main_element.get_attribute('outerHTML')
soup2 = BeautifulSoup(html, 'html.parser')

driver.quit()

## Pegando as combinações de moedas

In [ ]:
class_line = soup2.find_all('div', attrs={'class':"line"})
class_line

In [ ]:
combinacoes = []
for line in class_line:
    combinacoes.append(line.find("span").get_text().strip("<>"))


## Coletando as combinações por extenso

In [ ]:
combinacoes_extenso = []
for line in class_line:
    if len(line.find_all('span'))>1:
        combinacoes_extenso.append(str(unidecode.unidecode(line.find_all("span")[1].get_text())))

Criando um dicionário com todos os valores encontrados

In [ ]:
df_moedas = pandas.DataFrame(data = {"Cod_da_moeda": moedas, "Paises": paises})

In [ ]:
df_combinacoes = pandas.DataFrame(data={"Combinacoes_de_conversao":combinacoes, "Combinacoes_por_extenso": combinacoes_extenso})

In [ ]:
moedas_dict = {}
moedas_dict['Tipo da moeda'] = df_moedas.to_dict('records')

conversao_dict = {}
conversao_dict['Tipo de conversao'] = df_combinacoes.to_dict('records')

In [ ]:
combinacoes_extenso

## Convertendo em arquivo Json

In [ ]:
js = json.dumps(moedas_dict)
fp = open('moedas.json', 'w')
fp.write(js)
fp.close()

In [ ]:
js2 = json.dumps(conversao_dict)
fp = open('conversao.json', 'w')
fp.write(js2)
fp.close()

In [ ]:
import requests as r

#Pegar os tipos de moedas
dados_moedas = r.get("http://apimoeda.herokuapp.com/moedas")

#Pegar os tipos de conversões
dados_conversões =  r.get("http://apimoeda.herokuapp.com/conversao")

In [ ]:
import json
dict_moedas = dados_moedas.json()
dict_conversoes = dados_conversões.json()

In [ ]:
for i in dict_moedas['Tipo da moeda']:
    for key, value in i.items():
        print(key,value)

In [ ]:
for i in dict_conversoes['Tipo de conversao']:
    for key, value in i.items():
        print(key,value)

In [59]:
values = [value.values() for value in dict_moedas['Tipo da moeda']]

In [60]:
keys = [key.keys() for key in dict_moedas['Tipo da moeda']]

In [64]:
import pandas as pd
df = pd.DataFrame(values, index=keys, columns=['Cód Moeda', "País"])
print(df)

                       Cód Moeda                    País
(Cod_da_moeda, Paises)       AFN  Afghani do Afeganistao
(Cod_da_moeda, Paises)       MGA   Ariary Madagascarense
(Cod_da_moeda, Paises)       THB          Baht Tailandes
(Cod_da_moeda, Paises)       PAB        Balboa Panamenho
(Cod_da_moeda, Paises)       ETB             Birr Etiope
...                          ...                     ...
(Cod_da_moeda, Paises)      XAGG                  XPrata
(Cod_da_moeda, Paises)       XRP                     XRP
(Cod_da_moeda, Paises)       CNY             Yuan Chines
(Cod_da_moeda, Paises)       CNH    Yuan chines offshore
(Cod_da_moeda, Paises)       PLN           Zloti Polones

[155 rows x 2 columns]
